In [ ]:
import numpy as np # linear algebra
from matplotlib import pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
df_train = pd.read_csv('drfile.csv')
df_train.head(5)

In [ ]:
targets_series = pd.Series(df_train['diagnosis'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size = 512
x_train = []
y_train = []

i = 0 

for f, breed in tqdm(df_train.values):
    if type(cv2.imread('train/{}.png'.format(f)))==type(None):
        continue
    else:
        img = cv2.imread('train/{}.png'.format(f),0)
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
print('Done')

x_train = np.array(x_train, np.float32) / 255
y_train = np.array(y_train, np.uint8)

np.save('x_train2',x_train)
np.save('y_train2',y_train)

In [ ]:
x_train = np.load('x_train2.npy')
y_train = np.load('y_train2.npy')

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

In [ ]:
base_model = ResNet50V2(weights = None, include_top=False, input_shape=(512, 512,1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
# for layer in base_model.layers:
#     layer.trainable = False

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=3, validation_data=(X_valid, Y_valid), verbose=1)

In [ ]:
scores = model.evaluate(X_valid, Y_valid, verbose=2)

In [ ]:
model.save("DR.h5")